# Loading Responses

In [1]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("gaze-analysis/result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "glm-4v-9b" & df$list_id != -1 & df$Part != "p0", ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))

Loading required package: lme4

Loading required package: Matrix

Loading required package: readr

Loading required package: marginaleffects

Loading required package: emmeans

Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

Loading required package: car

Loading required package: carData

Rows: 156780 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Fitting models

In [2]:
model1 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0355467 (tol = 0.002, component 1)”


In [3]:
model2 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

boundary (singular) fit: see help('isSingular')



In [4]:
model3 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

# Model selection

In [5]:
model <- model3

# Interpretation

In [6]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Stimulus_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  6195.0   6244.8  -3090.5   6181.0     8993 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.99492 -0.13328 -0.06035  0.15027  3.09692 

Random effects:
 Groups      Name        Variance Std.Dev.
 Stimulus_ID (Intercept) 25.23    5.023   
Number of obs: 9000, groups:  Stimulus_ID, 900

Fixed effects:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          0.5260     0.3748   1.404  0.16043    
Angleleft                           -0.3161     0.4879  -0.648  0.51703    
Angleright                          -0.2926     0.4889  -0.598  0.54954    
scale(Proximity, scale = FALSE)     -0.725

In [7]:
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

term,Estimate,Std. Error,z value,Pr(>|z|),effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.5260132,0.3747527,1.4036275,1.604298e-01,5.340172,0.09850117
Angleleft,-0.3161462,0.4879364,-0.6479251,5.170334e-01,5.340172,-0.05920150
Angleright,-0.2925773,0.4888882,-0.5984545,5.495367e-01,5.340172,-0.05478800
"scale(Proximity, scale = FALSE)",-0.7257426,0.2489979,-2.9146534,3.560838e-03,5.340172,-0.13590248
"scale(n_candidates, scale = FALSE)",-1.4594303,0.2936401,-4.9701329,6.690701e-07,5.340172,-0.27329276
ActorY,-1.3476203,0.4176765,-3.2264689,1.253278e-03,5.340172,-0.25235523


In [8]:
avg_predictions(model, by="Angle", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(left) - (front),-0.034456034,0.04037645,-0.8533696,0.3934544,1.3457317,-0.11359242,0.04468036
(right) - (front),-0.025989962,0.03993535,-0.6508009,0.5151750,0.9568655,-0.10426181,0.05228189
(right) - (left),0.008466072,0.03126489,0.2707853,0.7865562,0.3463783,-0.05281199,0.06974413


In [9]:
avg_slopes(model, variables ="Proximity", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Proximity,dY/dX,-0.7257426,0.2490005,-2.914623,0.003561179,8.133429,-1.213775,-0.2377106,-0.3880239,-0.388169,-0.3880964


In [11]:
avg_slopes(model, variables ="n_candidates", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
n_candidates,dY/dX,-1.45943,0.2936426,-4.970091,6.692156e-07,20.51103,-2.034959,-0.8839014,-0.3879505,-0.3882424,-0.3880964


In [12]:
avg_predictions(model, by="Actor", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Y) - (X),-0.1010455,0.03026941,-3.338207,0.0008432096,10.21182,-0.1603725,-0.0417186
